In [6]:
from pyspark.sql import SparkSession

In [2]:
# change this URI for your account
horizon_catalog_uri = "https://org-account.snowflakecomputing.com/polaris/api/catalog"
CATALOG_NAME = "ICEBERG_TEST_DB"
SCHEMA_NAME = "DEMO"
ICEBERG_TABLE_NAME = "TEST_TABLE_EXTENDED"
pat_token               = ""
horizon_role        = "session:role:ICEBERG_DATA_ENGINEER"
region          = "eu-west-1" #aws ireland 

ICEBERG_VER = "1.9.1"

try:
    spark.stop()
except:
    pass

spark = (
    SparkSession.builder
    .master("local[*]")
    .config("spark.ui.port", "0")
    .config("spark.driver.bindAddress", "127.0.0.1")
    .config("spark.driver.host", "127.0.0.1")
    .config("spark.driver.port", "0")
    .config("spark.blockManager.port", "0")
    # 🔑 Pull the needed JARs automatically
    .config(
        "spark.jars.packages",
        f"org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:{ICEBERG_VER},"
        f"org.apache.iceberg:iceberg-aws-bundle:{ICEBERG_VER}"
    )
    # Hide UI progress
    .config("spark.ui.showConsoleProgress", "false")
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .config("spark.sql.defaultCatalog", "horizoncatalog")
    .config("spark.sql.catalog.horizoncatalog", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.horizoncatalog.type", "rest")
    .config("spark.sql.catalog.horizoncatalog.uri", horizon_catalog_uri)
    .config("spark.sql.catalog.horizoncatalog.warehouse", CATALOG_NAME)
    .config("spark.sql.catalog.horizoncatalog.header.X-Iceberg-Access-Delegation", "vended-credentials")
    .config("spark.sql.catalog.horizoncatalog.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.horizoncatalog.file-io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
    .config("spark.sql.catalog.horizoncatalog.client.region", region)
    # Use .credential for PAT. For External Oauth & key-pair auth use .token
    .config("spark.sql.catalog.horizoncatalog.credential", pat_token)
    .config("spark.sql.catalog.horizoncatalog.scope", horizon_role)
    .config("spark.sql.iceberg.vectorization.enabled", "false")
    .getOrCreate()
)

spark.sparkContext.setLogLevel("ERROR")

:: loading settings :: url = jar:file:/opt/anaconda3/envs/snowflake/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/ammarcic/.ivy2/cache
The jars for the packages stored in: /Users/ammarcic/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
org.apache.iceberg#iceberg-aws-bundle added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e5296cab-15ad-411c-98b4-f5412dbcd676;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.1 in central
	found org.apache.iceberg#iceberg-aws-bundle;1.9.1 in central
:: resolution report :: resolve 53ms :: artifacts dl 2ms
	:: modules in use:
	org.apache.iceberg#iceberg-aws-bundle;1.9.1 from central in [default]
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.9.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	--------------------------------------------

In [8]:
spark.sql("SHOW NAMESPACES").show(truncate=False)

+---------+
|namespace|
+---------+
|DEMO     |
|PUBLIC   |
+---------+



In [9]:
spark.sql(f"USE SCHEMA DEMO")

DataFrame[]

In [10]:
spark.sql(f"SELECT * FROM TEST_TABLE_EXTENDED").show(truncate=False)

+-------+----------+-------------------+
|COL_INT|COL_STRING|COL_TIMESTAMP_NTZ  |
+-------+----------+-------------------+
|1      |test      |2025-01-01 01:00:00|
|2      |test2     |2025-01-02 02:00:00|
|3      |test3     |2025-01-03 03:00:00|
+-------+----------+-------------------+

